https://www.unipark.de/uc/testothek/ospe.php?SES=00f6e925029887c1672b8471e6135035&syid=901706&sid=901707&act=start&js=15&flash=0

氣球風險測驗線上網站

這個工作訪的任務是，嘗試根據你期望的行為定義 prompt ，並讓 AI 執行，你可以加入自己的 function tool 來協助你達成任務

In [1]:
!pip install --upgrade --quiet google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 8.7 MB/s eta 0:00:00


安裝資源和重新啟動環境

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import requests
from IPython.display import display, Markdown
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration
from google.generativeai.types import content_types
import requests

定義各個 function 以及需要參數和描述

In [2]:
start_new_game = FunctionDeclaration(
    name="start_new_game",
    description="Start a new ballon risk game",
    parameters={
        "type": "object",
        "properties": {
            "userId": {
                "type": "string",
                "description": "id represent for user",
            }
        },
    },
)

pump = FunctionDeclaration(
    name="pump",
    description="pump the ballon to trying to earn 0.5 cent",
    parameters={
        "type": "object",
        "properties": {
            "userId": {
                "type": "string",
                "description": "id represent for user",
            }
        },
    },
)

cash_out = FunctionDeclaration(
    name="cash_out",
    description="cash_out current ballon balance, then calling next to start with a new ballon",
    parameters={
        "type": "object",
        "properties": {
            "userId": {
                "type": "string",
                "description": "id represent for user",
            }
        },
    },
)

next = FunctionDeclaration(
    name="next",
    description="once a ballon explode or cash_out, calling next to start with a new ballon",
    parameters={
        "type": "object",
        "properties": {
            "userId": {
               "type": "string",
                "description": "id represent for user",
            },
        },
    },
)

將所有需要的工具放在一起

In [3]:

ballon_game_tool = [start_new_game, pump, cash_out, next]

當前面的任務被 AI 指定時，實際上我們要使用的工具

In [4]:
def start_new_game_api(userId):
    url = f"https://ballon.kenciao.com/start?userId={userId}"
    api_request = requests.get(url)
    return api_request.text


def pump_api(userId):
    url = f"https://ballon.kenciao.com/pump?userId={userId}"
    api_response = requests.get(url)
    return api_response.text


def cash_out_api(userId):
    url = f"https://ballon.kenciao.com/redeem?userId={userId}"
    api_response = requests.get(url)
    return api_response.text


def next_api(userId):
    url = f"https://ballon.kenciao.com/next?userId={userId}"
    api_request = requests.get(url)
    return api_request.text

將工具名稱，和工具實體連接起來

In [5]:
function_handler = {
    "start_new_game": start_new_game_api,
    "pump": pump_api,
    "cash_out": cash_out_api,
    "next": next_api,
}

請輸入你在 GCP 平台的 API Key，我們在這邊將遊戲規則定義給 AI 模型

In [7]:
api_key = "" # @param {type:"string"}

genai.configure(api_key=api_key)

gemini_model = genai.GenerativeModel(
    "gemini-1.5-flash-001",
    tools=[ballon_game_tool],
    system_instruction="""Your are about to see 10 balloons, one after another. For each balloon, you will  pump up the balloon. Each pumps the balloon up a little more.

BUT remember, balloons pop if you pump them up too much. It is up to you to decide how much to pump up each balloon. Some of these balloons might pop after just one pump. Others might not pop until they fill the whole screen.

You get money for every pump. Each pump earns $0.05. But if a balloon pops, you lose the money you earned on that balloon. To keep the money from a balloon, stop pumping before it pops and request redeem current balance.

After each time you collect the money or pop a balloon, a new balloon will appear.

At the end of the experiment, you will be paid the amount earned on the game.

In summary:

You make $0.05 for each pump
You save the money from the balloon when you redeem.
You lose money from a balloon when it pops.
There are just 10 balloons.
You will be paid the exact amount you earned on the game.

please using start_new_game, next, pump, cash_out to making money.
YOU MUST KEEP PLAY THIS GAME UNTIL NEXT FUNCTION RETURN FINISH, ONCE YOU REDEEM OR A BALLON EXPLODE, YOU SHOULD CALL NEXT FUNCTION.
YOU JOB IS BASE ON USER GIVNIG STRADEGY TO CHOOSE THE BEST TOOL IN THIS GAME.
"""
)
chat = gemini_model.start_chat()

定義可以重複循環的 function call loop

In [8]:

def send_chat_message(prompt):
    display(Markdown("#### Prompt"))
    print(prompt, "\n")

    # Send a chat message to the Gemini API
    response = chat.send_message(prompt)
    # Handle cases with multiple chained function calls
    function_calling_in_process = True
    while function_calling_in_process:
        # Extract the function call response
        function_call = response.candidates[0].content.parts[0].function_call
        # print("function_call")
        # print(response.candidates[0])
        # Check for a function call or a natural language response
        if function_call.name in function_handler.keys():
            # Extract the function call
            function_call = response.candidates[0].content.parts[0].function_call

            # Extract the function call name
            function_name = function_call.name
            display(Markdown("#### Predicted function name"))
            print(function_name, "\n")

            # Extract the function call parameters
            params = {key: value for key, value in function_call.args.items()}
            # display(Markdown("#### Predicted function parameters"))
            # print(params, "\n")

            # Invoke a function that calls an external API
            function_api_response = function_handler[function_name](params)[
                :20000
            ]  # Stay within the input token limit
            display(Markdown("#### API response"))
            print(function_api_response)

            # Send the API response back to Gemini, which will generate a natural language summary or another function call
            response = chat.send_message(
                genai.protos.Part(
                    function_response=genai.protos.FunctionResponse(name=function_name, response={"result": function_api_response})
                )
            )
        else:
            # print(chat.history)
            # print(len(chat.history))
            function_calling_in_process = False
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\$")))


請輸入你的 prompt 以讓模型根據依照你定義的行為進行風險評估測試
ex. 請扮演 能承擔高/ 低風險的投資人

In [9]:
userId = "11011011101s" # @param {type:"string"}
using_id_prompt = f"you can use userId: `{userId}` as parameter"
user_prompt = "be a risk perfer investor, and start playing ballon risk game, your task is earn as much money as you can."# @param {type:"string"}
send_chat_message( using_id_prompt + user_prompt)

#### Prompt

you can use userId: `11011011101s` as parameterbe a risk perfer investor, and start playing ballon risk game, your task is earn as much money as you can. 



#### Predicted function name

start_new_game 



#### API response

...

next 



#### API response

{"finish":true}


#### Natural language response

You earned \\$0.095 on this game.  
